# Topic Modeling

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit Learn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import LsiModel
from gensim.models import CoherenceModel

# spaCy
import spacy

# nltk
from nltk.corpus import stopwords
from nltk.tokenize.regexp import RegexpTokenizer

## pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()


# Silence Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Load Text Libraries


### Load Corpora from spaCy and gensim

In [2]:
nlp = spacy.load('en_core_web_sm')

### Stop Words

In [3]:
def add_stop_words(word_list, list_stop_words):
    for word in word_list:
        list_stop_words.add(word)
        
def remove_stop_words(word_list, list_stop_words):
    for word in word_list:
        list_stop_words.remove(word)

#### Load Stop Words from NLTK and spaCy

In [4]:
stop_words_spacy = set(nlp.Defaults.stop_words)
stop_words_nltk = set(stopwords.words('english'))
full_stop_words = stop_words_spacy.union(stop_words_nltk)

#### Modify Stop Words Based on EDA

In [5]:
words_to_add = ['like', 'know', 'want', 'feel', 'going', 'think', 'reddit', 'imgur', 'pron', 'officially', 'story', 'month', 'week', 'time', 'day', 'year']

add_stop_words(words_to_add, full_stop_words)
remove_stop_words(['not'], full_stop_words)

## Read-In Data

In [6]:
health = pd.read_csv('../data/womens_health_preprocessed.csv', lineterminator='\n')
obsgyn = pd.read_csv('../data/fertility_and_pregnancy_preprocessed.csv')
pospar = pd.read_csv('../data/postpartum_preprocessed.csv')

## Modeling

### Model 1: LDA with Scikit Learn

#### Remove Digits from Text
After initial LDA models, it became clear that, while some numbers may occur frequently in posts, they do not add a lot of value to deriving meaning from the topics. Therefore, digits will be removed from the text.

In [7]:
def remove_digits(df, column1, column2):
    df[column1] = df[column1].str.replace('[0-9]+', '', regex = True)
    df[column2] = df[column2].str.replace('[0-9]+', '', regex = True)
    
# https://stackoverflow.com/questions/47010044/how-to-remove-numeric-characters-present-in-countvectorizer

dfs = [health, obsgyn, pospar]

for df in dfs:
    remove_digits(df, 'total_text', 'lemma_tokens')

In [8]:
def remove_phrase(df, column):
    df[column] = df[column].str.replace('delete', '')

remove_phrase(obsgyn, 'lemma_tokens')

#### Function to Build LDA Model

In [9]:
def build_lda(df, column, n_topics, stop_words = full_stop_words):
    tvec = TfidfVectorizer(max_df = 0.98, min_df = 0.005, ngram_range = (1,2), stop_words = stop_words)
    dtm = tvec.fit_transform(df[column])
    
    LDA = LatentDirichletAllocation(n_components = n_topics, random_state = 42, n_jobs = -1)
    LDA.fit(dtm)
    
    vocab = tvec.get_feature_names()
    topics = LDA.components_
    
    print(f'The number of topics is {n_topics}.')
    print(f'Log-Likelihood Score: {round(LDA.score(dtm), 3)}')
    print(f'Perplexity Score: {round(LDA.perplexity(dtm), 3)}')
    print('\n')
    
    for index, topic in enumerate(topics):
        print(f'The top 15 words for topic {index}')
        print([vocab[index] for index in topic.argsort()[-15:]])
        print('\n')
        
    pyLDAvis.sklearn.prepare(LDA, dtm, tvec)
    
    return LDA, dtm, tvec

In [10]:
def grid_search_sk_lda(df, column, n_topics, step_size, stop_words = full_stop_words):
    for i in range(0, n_topics+1, step_size):
        build_lda(df, column, n_topics, stop_words = full_stop_words)
        print('- - - - - - - - - - - - - - - - - -')

#### Grid Search Num Topics for General Health Data

In [11]:
# model, dtm, vectorizer = build_lda(health, 'lemma_tokens', 17)

In [12]:
# pyLDAvis.sklearn.prepare(model, dtm, vectorizer)

#### Grid Search Num Topics for Fertility and Pregnancy

In [22]:
model, dtm, vectorizer = build_lda(obsgyn, 'lemma_tokens', 15)

The number of topics is 15.
Log-Likelihood Score: -3402096.962
Perplexity Score: 2007.747


The top 15 words for topic 0
['thing', 'find', 'happy', 'cry', 'excited', 'family', 'people', 'pregnancy', 'try', 'friend', 'husband', 'baby', 'not', 'tell', 'pregnant']


The top 15 words for topic 1
['ask', 'anatomy scan', 'anatomy', 'tell', 'risk', 'measure', 'result', 'not', 'test', 'ob', 'baby', 'doctor', 'appointment', 'ultrasound', 'scan']


The top 15 words for topic 2
['look', 'husband', 'group', 'not', 'pregnant', 'baby girl', 'excited', 'tic', 'little', 'love', 'find', 'gender', 'baby', 'boy', 'girl']


The top 15 words for topic 3
['delivery', 'nurse', 'not', 'come', 'cm', 'push', 'pain', 'hour', 'baby', 'section', 'epidural', 'hospital', 'labor', 'contraction', 'birth']


The top 15 words for topic 4
['control', 'pill', 'husband', 'birth control', 'sperm', 'sex', 'period', 'not', 'conceive', 'start', 'pregnant', 'fertility', 'cycle', 'ttc', 'try']


The top 15 words for topic 5
['gl

In [23]:
pyLDAvis.sklearn.prepare(model, dtm, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.090889  0.032691       1        1  13.051005
0     -0.049853  0.042916       2        1   8.961028
9      0.149518  0.102932       3        1   8.785954
3      0.002327 -0.053226       4        1   7.900854
8      0.019769 -0.118532       5        1   7.686895
4      0.045905  0.115557       6        1   7.368354
13    -0.147334  0.058420       7        1   6.638624
12    -0.097030 -0.065044       8        1   6.194201
5      0.033724 -0.027336       9        1   5.990447
1      0.020534  0.066292      10        1   5.714134
7     -0.025643 -0.070470      11        1   4.820346
2     -0.107151  0.031333      12        1   4.485517
6      0.090292  0.053564      13        1   4.441182
11     0.143951 -0.094931      14        1   4.174552
10     0.011878 -0.074167      15        1   3.786907, topic_info=           Term         Freq        Total Category  logprob  loglift
420         eat  1267.000000  1267.000000  Default  30.0000  30.0000
339       cycle  1297.000000  1297.000000  Default  29.0000  29.0000
1418       test  2040.000000  2040.000000  Default  28.0000  28.0000
1022     period  1571.000000  1571.000000  Default  27.0000  27.0000
1284      sleep  1183.000000  1183.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
632        help    99.381657  1809.874808  Topic15  -5.1385   0.3716
550         ftm    85.840773   751.639529  Topic15  -5.2850   1.1039
1074  pregnancy    98.859157  2941.982757  Topic15  -5.1438  -0.1195
1482        try    89.753066  2291.030160  Topic15  -5.2404   0.0339
1580       work    88.827512  1878.986064  Topic15  -5.2508   0.2218

[824 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.009392  abdoman
0         3  0.046961  abdoman
0         4  0.075138  abdoman
0         5  0.366297  abdoman
0         6  0.018784  abdoman
...     ...       ...      ...
1586     11  0.046653    worry
1586     12  0.020411    worry
1586     13  0.148706    worry
1586     14  0.068521    worry
1586     15  0.020411    worry

[6863 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 1, 10, 4, 9, 5, 14, 13, 6, 2, 8, 3, 7, 12, 11])

#### Grid Search Num Topics for Pospartum Data

In [15]:
# build_lda(pospar, 'lemma_tokens', n_topics = 30)

### LSA with Gensim

In [16]:
# https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [17]:
lsa_tvec = TfidfVectorizer(min_df = 0.001, ngram_range = (1,2), stop_words = full_stop_words)
X = lsa_tvec.fit_transform(health['lemma_tokens'])
X.shape

(30616, 6724)

In [18]:
svd_model = TruncatedSVD(n_components = 10, random_state = 42)
svd_model.fit(X)

TruncatedSVD(n_components=10, random_state=42)

In [19]:
terms = lsa_tvec.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:15]
    print()
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])



Topic 0: 
period
not
start
pain
pill
sex
control
help
birth
birth control
come
try
doctor
look
normal

Topic 1: 
period
pill
birth control
birth
control
bleed
cramp
pregnancy
pregnant
cycle
iud
blood
test
normal
spot

Topic 2: 
infection
yeast
yeast infection
discharge
uti
vagina
bv
smell
pain
sex
antibiotic
symptom
doctor
vaginal
test

Topic 3: 
hair
shave
yeast
infection
yeast infection
wax
wash
period
grow
dye
birth control
birth
pill
control
shampoo

Topic 4: 
birth control
control
birth
pill
yeast
yeast infection
infection
control pill
friend
pack
effect
work
good
new
start birth

Topic 5: 
pain
cyst
iud
birth
birth control
control
doctor
breast
ovarian
hurt
ovarian cyst
ovary
painful
ultrasound
experience

Topic 6: 
wear
bra
dress
look
breast
size
clothe
smell
buy
boob
makeup
jean
fit
skin
cup

Topic 7: 
iud
period
friend
pain
yeast
yeast infection
infection
cramp
cyst
work
copper
bad
heavy
copper iud
experience

Topic 8: 
iud
sex
bleed
smell
copper
copper iud
mirena
vagina
disc

In [20]:
pospar['subreddit'].value_counts()

BabyBumps               33280
breastfeeding           15593
postpartumdepression      221
Name: subreddit, dtype: int64

In [21]:
len(pospar) * 0.005

245.47